# Link Prediction and Retrieval on Graphs  with Adamic Adar and Common Neighbours


In [2]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict
from torch_geometric.utils import to_undirected, negative_sampling, subgraph

#  CoraFull Complete Dataset

In [4]:
from torch_geometric.datasets import CoraFull

dataset = CoraFull(root='path/to/data/CoraFull')
data = dataset[0]

edge_index = to_undirected(data.edge_index) # directed or undirected?;  undirected

num_edges = edge_index.size(1)
num_nodes = data.num_nodes

train_size = int(0.6* num_edges)
test_size = num_edges - train_size

train_edge_index = edge_index[:, :train_size]
test_edge_index = edge_index[:, train_size:]

train_nodes= train_edge_index.reshape(-1).unique()
test_nodes = test_edge_index.reshape(-1).unique()

print("CoraFull ",data) # how do we make use of y; no need here
print(f'nodes: {data.num_nodes};   features per node: {data.num_node_features};  classes: {dataset.num_classes};')
print()
print(f'edges: {data.num_edges}    train_edges: {train_edge_index.size(1)}      test_edges: {test_edge_index.size(1)} ')
print(f"train_nodes: {train_nodes.size(0)}     test_nodes: {test_nodes.size(0)}")

Processing...
Done!


CoraFull  Data(x=[19793, 8710], edge_index=[2, 126842], y=[19793])
nodes: 19793;   features per node: 8710;  classes: 70;

edges: 126842    train_edges: 76105      test_edges: 50737 
train_nodes: 17426     test_nodes: 16178


# Q - Test Nodes

In [5]:
def get_connected_nodes(node_id, our_edge_index):
    mask_u = our_edge_index[0] == node_id
    mask_v = our_edge_index[1] == node_id
    neighbors_u = our_edge_index[1][mask_u] 
    neighbors_v = our_edge_index[0][mask_v]  
    connected_nodes = torch.cat([neighbors_u, neighbors_v])
    connected_nodes = torch.unique(connected_nodes)
    return connected_nodes

#finding Q set
triangle_nodes = set() 
test_edges = test_edge_index.t() 
print("test_edges:",test_edges.size(), "test_nodes:",len(test_nodes))
start_time = time.time()
for u, v in test_edges:
    nodes_u=get_connected_nodes(u, test_edge_index)
    nodes_v=get_connected_nodes(v, test_edge_index)
    mask = torch.isin(nodes_u, nodes_v)
    intersection_nodes = nodes_u[mask]
    if intersection_nodes.size(0) > 1:
        triangle_nodes.add(u.item())
        triangle_nodes.add(v.item())
triangle_nodes_tensor = torch.tensor(list(triangle_nodes), dtype=torch.long)
Q = triangle_nodes_tensor[:1000]
end_time = time.time()
Qtime=end_time-start_time
print("triangle_nodes:",len(triangle_nodes), f"; time taken to find them:{Qtime:.8f} seconds")
print("(test nodes that are part of atleast one triangle in test induced subgraph)")
print()
print(f"Q set, first {len(Q)} nodes of triangle_nodes")
print("Q:",Q)

test_edges: torch.Size([50737, 2]) test_nodes: 16178
triangle_nodes: 4835 ; time taken to find them:26.29346728 seconds
(test nodes that are part of atleast one triangle in test induced subgraph)

Q set, first 1000 nodes of triangle_nodes
Q: tensor([ 8192, 16386,  8195,  8196,  8197, 16388, 16389, 16392, 16393, 16390,
            9, 16396,    12, 16398,  8207,  8208, 16401, 16402, 16403,  3260,
        16405,  8213, 16407,  8214, 16406,  8218, 16408, 16404,  8221,    30,
        16415,    31, 16409, 16410, 16420, 16422,    39, 16423, 16425, 16426,
        16427, 16424, 16431, 16432, 16433, 16434, 16436, 16439, 16441, 16446,
        16449, 16450,  8259, 16451,    68, 16454, 16453, 16456, 16457, 16458,
        16459,    76, 16452, 16455, 16466, 16467,    82,    85, 16471, 16473,
        16474,  8282, 16476,    91, 16478, 16481, 16482, 16483, 16484, 16486,
        16491, 16492, 19560, 16494, 16495, 16493, 16498, 16499, 16500, 19562,
        16503, 16504,  8313, 16511, 16512, 16513, 16515,

# Adamic Adar

In [12]:
import math

def adamic_adar_score(u, v, edge_index):
    u_neighbours = get_connected_nodes(u, edge_index)
    scores = []
    for node in v:
        v_neighbours = get_connected_nodes(node, edge_index)
        common_neighbors = u_neighbours[torch.isin(u_neighbours, v_neighbours)]
        score = 0.0
        for neighbor in common_neighbors:
            neighbor_degree = len(get_connected_nodes(neighbor, edge_index))
            if neighbor_degree > 1: 
                score += 1 / math.log(neighbor_degree)       
        scores.append(score)   
    return torch.tensor(scores)

def evaluate_adamicadar(Q, test_nodes, edge_index, k_list):
    times=0
    results = {}
    precision = {q.item(): {} for q in Q}
    precision_avg = {k: 0.0 for k in k_list}
    mrr = {q.item(): 0.0 for q in Q} 
    mrr_avg = 0.0 
    i = 0
    for q in Q:
        i = i + 1
        start_time = time.time()
        adamicadar_scores = adamic_adar_score(q, test_nodes, edge_index)
        sorted_scores, sorted_indices = torch.sort(adamicadar_scores, descending=True)
        sorted_nodes = [test_nodes[i] for i in sorted_indices.tolist()]
        sorted_nodes= [node for node in sorted_nodes if node != q]
        results[q.item()] = sorted_nodes
        end_time = time.time()
        times=times+end_time-start_time
        q_neighbours = get_connected_nodes(q, edge_index)
        for k in k_list:
            top_k_nodes = sorted_nodes[:k]
            intersection_length = torch.sum(torch.isin(torch.tensor(top_k_nodes), q_neighbours)).item()
            precision[q.item()][k] = intersection_length / k 
            precision_avg[k] += precision[q.item()][k]
        rank = -1
        for idx, node in enumerate(sorted_nodes):
            if node in q_neighbours:
                rank = idx + 1  
                break
        if rank != -1:
            mrr[q.item()] = 1 / rank  
        mrr_avg += mrr[q.item()]  
        if(i%10==0):
            print(i)
    for k in k_list:
        precision_avg[k] /= len(Q)
    times=times/len(Q)  
    mrr_avg /= len(Q)
    return results, precision, precision_avg, times, mrr_avg

k_list=[1,5,10]
adamicadar_results, precision_results, precision_avg, times, mrr_avg = evaluate_adamicadar(Q, test_nodes, edge_index, k_list)
print(f"ADAMIC ADAR; CORAFULL dataset;")
print()
for k, avg_precision in precision_avg.items():
        print(f"P@{k}: {avg_precision:.4f}")
print("MRR: ",mrr_avg)
print(f"Retrieval Time:{times:.8f} seconds")

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
ADAMIC ADAR; CORAFULL dataset;

P@1: 0.6510
P@5: 0.5296
P@10: 0.4318
MRR:  0.7741454204567364
Retrieval Time:8.21797911 seconds


# **Common Neighbours**

In [6]:
import math

def common_neighbour_score(q, test_nodes, edge_index):
    q_neighbours = get_connected_nodes(q, edge_index)
    cn_scores = []
    for node in test_nodes:
        node_neighbours = get_connected_nodes(node, edge_index)
        common_neighbors = torch.isin(q_neighbours, node_neighbours).sum().item()
        cn_scores.append(common_neighbors)
    return torch.tensor(cn_scores)

def evaluate_common_neighbour(Q, test_nodes, edge_index, k_list):
    times=0
    results = {}
    precision = {q.item(): {} for q in Q}
    precision_avg = {k: 0.0 for k in k_list}
    mrr = {q.item(): 0.0 for q in Q} 
    mrr_avg = 0.0 
    i = 0
    for q in Q:
        i = i + 1
        start_time = time.time()
        common_neighbour_scores = common_neighbour_score(q, test_nodes, edge_index)
        sorted_scores, sorted_indices = torch.sort(common_neighbour_scores, descending=True)
        sorted_nodes = [test_nodes[i] for i in sorted_indices.tolist()]
        sorted_nodes= [node for node in sorted_nodes if node != q]
        results[q.item()] = sorted_nodes
        end_time = time.time()
        times=times+end_time-start_time
        q_neighbours = get_connected_nodes(q, edge_index)
        for k in k_list:
            top_k_nodes = sorted_nodes[:k]
            intersection_length = torch.sum(torch.isin(torch.tensor(top_k_nodes), q_neighbours)).item()
            precision[q.item()][k] = intersection_length / k 
            precision_avg[k] += precision[q.item()][k]
        rank = -1
        for idx, node in enumerate(sorted_nodes):
            if node in q_neighbours:
                rank = idx + 1  
                break
        if rank != -1:
            mrr[q.item()] = 1 / rank  
        mrr_avg += mrr[q.item()]  
        if(i%10==0):
            print(i)
    for k in k_list:
        precision_avg[k] /= len(Q)
    times=times/len(Q)  
    mrr_avg /= len(Q)
    return results, precision, precision_avg, times, mrr_avg

k_list=[1,5,10]
common_neighbour_results, precision_results, precision_avg, times, mrr_avg = evaluate_common_neighbour(Q, test_nodes, edge_index, k_list)
print(f"COMMON NEIGHBOUR; CORAFULL dataset;")
print()
for k, avg_precision in precision_avg.items():
        print(f"P@{k}: {avg_precision:.4f}")
print("MRR: ",mrr_avg)
print(f"Retrieval Time:{times:.8f} seconds")

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
COMMON NEIGHBOUR; CORAFULL dataset;

P@1: 0.5820
P@5: 0.5002
P@10: 0.4107
MRR:  0.7194956217656225
Retrieval Time:8.30217773 seconds
